In [4]:
from main import load_and_preprocess_data

df = load_and_preprocess_data(1)

Loading and preprocessing data...
Class distribution:
is_fraud
0    1289169
1       7506
Name: count, dtype: int64
Fraud percentage: 0.58%


In [7]:
from model import load_lstm_model
from predict import predict_and_analyze
from main import SEQUENCE_LENGTH

sample_cc_num = df[df['is_fraud'] == 1]['cc_num'].iloc[0]
user_transactions = df[df['cc_num'] == sample_cc_num]

user_non_fraud_transactions = user_transactions[user_transactions['is_fraud'] == 0].iloc[:-1]
user_fraud_transactions = user_transactions[user_transactions['is_fraud'] == 1]

last_transaction = user_transactions.iloc[-1].to_dict()

lstm_model, processor = load_lstm_model()

results = predict_and_analyze(
    lstm_model,
    user_non_fraud_transactions,
    SEQUENCE_LENGTH,
    processor,
    actual_transaction=last_transaction,
    fraud_transactions=user_fraud_transactions
)

print("\nPredicted most likely values for next transaction:")
print("---------------------------------------------")
for feature, value in results['prediction']['most_likely_values'].items():
    print(f"  {feature}: {value}")
            

print('\nReal values:')
print("---------------------------------------------")
for feature in results['prediction']['most_likely_values'].keys():
    print(f"  {feature}: {last_transaction[feature]}")

print('\nFeature probability:')
print("---------------------------------------------")
for feature, probability in results['feature_probabilities'].items():
    print(f"  {feature}: {probability}")

print('\nTotal probability:')
print("---------------------------------------------")
print(results['total_probability'])

Loading model from models/lstm_transaction_model.h5...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step

Predicted most likely values for next transaction:
---------------------------------------------
  amt: {'lower_bound': np.float64(6.17), 'upper_bound': np.float64(6.93), 'avg': np.float64(6.55)}
  lat: {'lower_bound': np.float64(42.6315), 'upper_bound': np.float64(43.033), 'avg': np.float64(42.83225)}
  long: {'lower_bound': np.float64(-112.4777), 'upper_bound': np.float64(-105.6933), 'avg': np.float64(-109.0855)}
  merch_lat: {'lower_bound': np.float64(42.71336933333333), 'upper_bound': np.float64(43.385511466666664), 'avg': np.float64(43.049440399999995)}
  merch_long: {'lower_bound': np.float64(-111.96667829999998), 'upper_bound': np.float64(-105.2959852), 'avg': np.float64(-108.63133174999999)}
  hour: {'avg': np.int64(12)}
  merchant_encoded: {'avg': np.int64(508)}
  category_encoded: {'avg': np.int64(2)}

Real values:
---------------------------------------------
  amt: 61.93
  lat: 43